## Softmax classifier approach
---
This approach assumes that quantifiers are learned as a group and that essentially each q quantifier example is a negative example for all other quantifiers q'.

The classifier is in effect a solver for which q makes the sentence "Q as are bs" most likely given an input scene s.

This enables us to use not onlt the quantifier quantify evaluation methods but the classifier in order to generate a teacher-student scheme.

## Imports

### my class imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from quants import *
from models import Classifier

Using TensorFlow backend.


### Global imports

In [3]:
import numpy as np
import pandas as pd

### keras and TF imports

In [4]:
import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

import keras

from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Embedding, Dense, Conv1D, Input, Bidirectional, RepeatVector, Dropout, LeakyReLU, Flatten
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import SGD, Adam

TensorFlow version:  2.2.0


In [5]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.1)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
print("Keras backend: ", tf.python.keras.backend.backend())
tf.python.keras.backend.set_session(sess)
tf.config.list_logical_devices()


Keras backend:  tensorflow


[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU')]

In [6]:
# from functools import partial, update_wrapper

# def wrapped_partial(func, *args, **kwargs):
#     |   partial_func = partial(func, *args, **kwargs)
#         update_wrapper(partial_func, func)
#         return partial_func

### Classifier models

In [7]:
# deep dense classifier model builder method
def DDNNBuilder(quantifiers):
    model= Sequential()
    model.add(Dense(scene_len, activation="relu", name="input"))
    model.add(Dropout(0.25, name="dropout_1"))
    model.add(Dense(100, activation="relu", name="dense_2"))
    model.add(Dropout(0.25, name="dropout_2"))
    model.add(Dense(50, activation="relu", name="dense_3"))
    model.add(Dropout(0.25, name="dropout_3"))
    model.add(Dense(len(quantifiers), activation='softmax', name="softmax_1"))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                              tf.keras.metrics.Recall()])
    return model, False

In [8]:
# dense classifier model builder method
def DNNBuilder(quantifiers):
    model= Sequential()
    model.add(Dense(scene_len, activation="relu", name="input"))
    model.add(Dropout(0.5, name="dropout_1"))
    model.add(Dense(len(quantifiers), activation='softmax', name="softmax_1"))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                              tf.keras.metrics.Recall()])
    return model, False

In [9]:
from tensorflow.keras import initializers

# Convolutional classifier model builder method
def CNNBuilder(quantifiers):
    model= Sequential()
    model.add(Conv1D(filters=2, kernel_size=1, 
                     use_bias=False, 
                     input_shape=(scene_len, len(symbols)), name="conv_1"))
    model.add(Dropout(0.5, name="dropout_1"))
    model.add(Flatten())
    model.add(Dense(len(quantifiers),
#                     kernel_initializer="constant", trainable=False, use_bias=False, 
                    activation='softmax', name="softmax_1"))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision(),
                                                                              tf.keras.metrics.Recall()])
    return model, True

## Quantifier sets for learning

In [10]:
natural_quantifiers = [The(), Both(), No(), All(), Some(), Most()]

In [11]:
unnatural_quantifiers = [MinMax(2, 10), MinMax(3, 6), Or([MinMax(2, 5), MinMax(10, 20)])]
# unnatural_quantifiers = [MinMax(2, 5), MinMax(8, 10), MinMax(12, 15), MinMax(17, 20), MinMax(24, 30), MinMax(37, 50)]

In [62]:
def teach(classifier, min_len=0, max_len=scene_len, repeat=1, epochs=50, batch_size=10):
    """
    This method teaches a classifier to classify its quantifiers
    
    repeat: teacher student learning for repeat # of rounds
    epochs, batch_size: parameters passed to tensorflow learning
    min_len, max_len: genereated scene length limits for training (to test generalization)
    """
    last_classifier = None
    with tf.device("/cpu:0"):
#     with tf.device("/gpu:0"):
        # iterate while using the previous model as label generator
        for _ in range(repeat):
            # generate fit and test model
            if last_classifier:
                train_scenes_labels = classifier.generate_labeled_scenes(last_classifier, min_len, max_len)
                test_scenes_labels = classifier.generate_labeled_scenes(last_classifier)
            else:
                train_scenes_labels = classifier.generate_labeled_scenes(min_len, max_len)
                test_scenes_labels = classifier.generate_labeled_scenes()
            classifier.fit(*train_scenes_labels, epochs=epochs, batch_size=batch_size)
            classifier.test(*test_scenes_labels)
            classifier.test_random(1000)
            last_classifier = classifier.clone()
        return classifier

In [63]:
natural_classifier = teach(Classifier(natural_quantifiers, CNNBuilder), epochs=50, max_len=100)
# natural_classifier = teach(Classifier(natural_quantifiers, DNNBuilder), epochs=500, repeat=3)

CNNBuilder model classifies ['All()' 'Both()' 'Most()' 'No()' 'Some()' 'The()']
Epoch 1/50
600/600 [==============================] - 0s 198us/step - loss: 1.8198 - precision_8: 0.2298 - recall_8: 0.1184
Epoch 2/50
600/600 [==============================] - 0s 88us/step - loss: 1.4717 - precision_8: 0.3465 - recall_8: 0.1966
Epoch 3/50
600/600 [==============================] - 0s 82us/step - loss: 1.4111 - precision_8: 0.3849 - recall_8: 0.2180
Epoch 4/50
600/600 [==============================] - 0s 82us/step - loss: 1.2406 - precision_8: 0.4116 - recall_8: 0.2344
Epoch 5/50
600/600 [==============================] - 0s 81us/step - loss: 1.2684 - precision_8: 0.4383 - recall_8: 0.2530
Epoch 6/50
600/600 [==============================] - 0s 82us/step - loss: 1.1360 - precision_8: 0.4506 - recall_8: 0.2629
Epoch 7/50
600/600 [==============================] - 0s 81us/step - loss: 1.0467 - precision_8: 0.4801 - recall_8: 0.2868
Epoch 8/50
600/600 [==============================] - 0s 8

In [67]:
# unnatural_model = teach(Classifier(unnatural_quantifiers, CNNBuilder), epochs=50, max_len=100)
unnatural_model = teach(Classifier(unnatural_quantifiers, DNNBuilder), epochs=50, max_len=100)

DNNBuilder model classifies ['MinMax(m=2,M=10)' 'MinMax(m=3,M=6)'
 'Or(MinMax(m=2,M=5),MinMax(m=10,M=20))']
Epoch 1/50
300/300 [==============================] - 0s 478us/step - loss: 3.0564 - precision_11: 0.3747 - recall_11: 0.3629
Epoch 2/50
300/300 [==============================] - 0s 260us/step - loss: 1.8069 - precision_11: 0.3828 - recall_11: 0.3511
Epoch 3/50
300/300 [==============================] - 0s 262us/step - loss: 1.3043 - precision_11: 0.3854 - recall_11: 0.3332
Epoch 4/50
300/300 [==============================] - 0s 268us/step - loss: 1.1271 - precision_11: 0.3816 - recall_11: 0.2793
Epoch 5/50
300/300 [==============================] - 0s 243us/step - loss: 1.0762 - precision_11: 0.3861 - recall_11: 0.2412
Epoch 6/50
300/300 [==============================] - 0s 239us/step - loss: 1.0821 - precision_11: 0.3935 - recall_11: 0.2168
Epoch 7/50
300/300 [==============================] - 0s 245us/step - loss: 1.0622 - precision_11: 0.3984 - recall_11: 0.2021
Epoch 8/50